In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [2]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib tk

In [3]:
import os, os.path, sys

In [4]:
modelMSE = keras.saving.load_model(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\modeli\Difference predictions\Y - CNN - simple")

In [5]:
path = r"C:\Users\mata2\Desktop\master\podaci\novi_deo\y-0k"

In [6]:
dir = os.listdir(path)

Ucitavanje fajlova

In [7]:
file_names = []
for file in dir:
    file_names.append(file)

In [8]:
test_file_names = file_names

Ucitavanje zadate trajektorije

In [9]:
y_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\novi_deo\yCmd_novi_deo.txt",header=None).astype('int'))

In [10]:
sig_len = len(y_command)

Konkatenacija fajlova koji se posle pravilno rasporedjuju na adekvatne train/test skupove

In [11]:
test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:len(y_command)].astype('int')
                              for file in test_file_names], axis=0)

Prethodno konkatenizovani podaci se adekvatno dele kako bi se izbeglo fiktivno stanje (stanje pri kraju rada i pocetku novog ako su konkatenizovani podaci)

In [12]:
for j in range(len(test_file_names)):
    exec(f"razlika_test_{j} = []")
    for i in range(len(y_command)):
        exec(f"razlika_test_{j}.append(y_command[i] - test_[j*len(y_command)+i])")
    exec(f'razlika_test_{j} = np.asarray(razlika_test_{j})')    

Funkcija koja deli podatke na "sliding window"

In [14]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

Buffer

In [13]:
time_steps = 10

Deljenje podataka na train i test skup

In [15]:
for i in range(len(test_file_names)):
    exec(f'test_X_{i}, test_y_{i} = create_timeseries_split(razlika_test_{i}, time_steps)')

test_X = np.asarray(test_X_0)
for i in range(1,len(test_file_names)):
    exec(f'test_X = np.append(test_X, test_X_{i}, axis=0)')

test_y = np.asarray(test_y_0)
for i in range(1,len(test_file_names)):
    exec(f'test_y = np.append(test_y, test_y_{i}, axis=0)')

In [16]:
input_shape = (time_steps,1)

In [17]:
#train_predict_MSE = modelMSE.predict(train_X)
test_predict_MSE = modelMSE.predict(test_X)

16871/16871 [==============================] - 22s 1ms/step


Ucitavamo standardne devijacije i srednje vrednosti

In [19]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k - y novi podaci\0k - y novi podaci_std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [18]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k - y novi podaci\0k - y novi podaci_sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

In [20]:
mean_1std = 0
for j in range(len(test_file_names)):
    for i in range(int(len(test_predict_MSE)/len(test_file_names))):
        if sr_vrednost[i+time_steps] - std_dev[i+time_steps] <= test_predict_MSE[int(len(test_predict_MSE)/len(test_file_names))*j + i] <= sr_vrednost[i+time_steps] + std_dev[i+time_steps]:
            mean_1std += 1
        #else:
            #print(sr_vrednost[i+8], std_dev[i+8], test_predict_MSE[60992*j + i], 'index:',j*60992 + i)
mean_1std/len(test_predict_MSE)

0.6534926368435676

Mean 3 sigma

In [21]:
i = time_steps
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(y_command) == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
mean_3std/len(test_predict_MSE)

0.9818097619709178

In [34]:
i = time_steps
uspeli_minmax = 0
for j in range(len(test_predict_MSE)):
    if i%sig_len == 0:
        i = time_steps
    if min_kord[i] <= (test_predict_MSE[j]) <= max_kord[i]:
        uspeli_minmax += 1
    i += 1
uspeli_minmax/len(test_predict_MSE)

0.8618838566268407

# NA TEST SKUPU - plotovi trajektorija

test_ = realizovana trajektorija

test_predict_MSE = predvidjena RAZLIKA koordinata od y_command

razlika_test_{} = realizovana RAZLIKA od test podataka od y_command - CELA TRAJEKTORIJA

test_y_{} = REALIZOVANA y koordinata od razlika_test_{}

Y_COMMAND - TEST_ = ODSTUPANJE/RAZLIKA_TEST_

Y_COMMAND[TIME_STEPS] + test_predict_MSE[0] = PRVU KOORDINATU TRAJEKTORIJE PREDVIDJENE VREDNOSTI

In [21]:
tri_sigma_lower = []
tri_sigma_upper = []

for i in range(len(y_command)):
    tri_sigma_lower.append(sr_vrednost[i] - 3*std_dev[i])
    tri_sigma_upper.append(sr_vrednost[i] + 3*std_dev[i])

In [22]:
for j in range(len(test_file_names)):
    exec(f'trajektorija_odstupanja_{j} = []')
    for i in range(sig_len - time_steps):
        exec(f'trajektorija_odstupanja_{j}.append(test_predict_MSE[j*(sig_len-time_steps) + i])')

In [24]:
trajektorija = 1
i = time_steps
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(y_command) == 0:
        print("trajektorija ",trajektorija, " sadrzi ", mean_3std , " tacaka", ", procenat tri sigma iznosi: ",round(mean_3std/36000*100,2),"%")
        exec(f"trajektorija_{trajektorija} = mean_3std")
        trajektorija += 1
        mean_3std = 0
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
trajektorija_15 = mean_3std
print("trajektorija ",trajektorija, " sadrzi ", mean_3std , " tacaka", ", procenat tri sigma iznosi: ",round(mean_3std/36000*100,2),"%")

trajektorija  1  sadrzi  35520  tacaka , procenat tri sigma iznosi:  98.67 %
trajektorija  2  sadrzi  35119  tacaka , procenat tri sigma iznosi:  97.55 %
trajektorija  3  sadrzi  35461  tacaka , procenat tri sigma iznosi:  98.5 %
trajektorija  4  sadrzi  35316  tacaka , procenat tri sigma iznosi:  98.1 %
trajektorija  5  sadrzi  35470  tacaka , procenat tri sigma iznosi:  98.53 %
trajektorija  6  sadrzi  35428  tacaka , procenat tri sigma iznosi:  98.41 %
trajektorija  7  sadrzi  35516  tacaka , procenat tri sigma iznosi:  98.66 %
trajektorija  8  sadrzi  35469  tacaka , procenat tri sigma iznosi:  98.52 %
trajektorija  9  sadrzi  35360  tacaka , procenat tri sigma iznosi:  98.22 %
trajektorija  10  sadrzi  35237  tacaka , procenat tri sigma iznosi:  97.88 %
trajektorija  11  sadrzi  34626  tacaka , procenat tri sigma iznosi:  96.18 %
trajektorija  12  sadrzi  35410  tacaka , procenat tri sigma iznosi:  98.36 %
trajektorija  13  sadrzi  35448  tacaka , procenat tri sigma iznosi:  98.47

In [25]:
plt.figure(figsize=(11,7))
plt.plot(tri_sigma_lower[time_steps:], color= 'b')
plt.plot(tri_sigma_upper[time_steps:], color = 'r')
plt.plot(trajektorija_odstupanja_0, color = 'g')
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.legend(['три сигма доња граница',"три сигма горња граница", "одступање трајекторије 1"],loc = 'best')
plt.show()

In [26]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_0)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_0[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_0[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_0[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_1/36000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [27]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_1)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_1[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_1[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_1[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_2/36000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [22]:
min_kord = [float('inf') for x in range(len(y_command))]
max_kord = [float('-inf') for x in range(len(y_command))]

for i in range(len(y_command)):    
    for j in range(len(test_file_names)):
        exec(f'current_file = razlika_test_{j}')
        if current_file[i][0] < min_kord[i]:
            min_kord[i] = current_file[i][0]
        if current_file[i][0] > max_kord[i]:
            max_kord[i] = current_file[i][0]

In [23]:
min_trajektorija = []
max_trajektorija = []
for i in range(len(y_command)):
    min_trajektorija.append(y_command[i] + min_kord[i])
    max_trajektorija.append(y_command[i] + max_kord[i])

In [20]:
for j in range(len(test_file_names)):
    exec(f'predvidjena_trajektorija_{j} = []')
    for i in range(len(y_command) - time_steps):
        exec(f'predvidjena_trajektorija_{j}.append(y_command[i + time_steps] + test_predict_MSE[j*(len(y_command)-time_steps) + i])')

In [24]:
trajektorija_koord = 1
i = time_steps
mean_3std_koord = 0
for j in range(len(test_predict_MSE)):
    if i%len(y_command) == 0:
        print("trajektorija ",trajektorija_koord, " sadrzi ", mean_3std_koord , " tacaka", ", procenat tri sigma iznosi: ",round(mean_3std_koord/36000*100,2), "%")
        exec(f"trajektorija_koord_{trajektorija_koord} = mean_3std_koord")
        trajektorija_koord += 1
        mean_3std_koord = 0
        i = time_steps
    if min_kord[i] <= (test_predict_MSE[j]) <= max_kord[i]:
            mean_3std_koord += 1
    i += 1
trajektorija_koord_15 = mean_3std_koord
print("trajektorija ",trajektorija_koord, " sadrzi ", mean_3std_koord , " tacaka", ", procenat tri sigma iznosi: ",round(mean_3std_koord/36000*100,2),"%")

trajektorija  1  sadrzi  32105  tacaka , procenat tri sigma iznosi:  89.18 %
trajektorija  2  sadrzi  31976  tacaka , procenat tri sigma iznosi:  88.82 %
trajektorija  3  sadrzi  31539  tacaka , procenat tri sigma iznosi:  87.61 %
trajektorija  4  sadrzi  31635  tacaka , procenat tri sigma iznosi:  87.88 %
trajektorija  5  sadrzi  30344  tacaka , procenat tri sigma iznosi:  84.29 %
trajektorija  6  sadrzi  31095  tacaka , procenat tri sigma iznosi:  86.38 %
trajektorija  7  sadrzi  32198  tacaka , procenat tri sigma iznosi:  89.44 %
trajektorija  8  sadrzi  31442  tacaka , procenat tri sigma iznosi:  87.34 %
trajektorija  9  sadrzi  30752  tacaka , procenat tri sigma iznosi:  85.42 %
trajektorija  10  sadrzi  31115  tacaka , procenat tri sigma iznosi:  86.43 %
trajektorija  11  sadrzi  29733  tacaka , procenat tri sigma iznosi:  82.59 %
trajektorija  12  sadrzi  30126  tacaka , procenat tri sigma iznosi:  83.68 %
trajektorija  13  sadrzi  31590  tacaka , procenat tri sigma iznosi:  87.

In [25]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_0)):
    if i%36000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=2)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин макс појас износи: {}%'.format(round((trajektorija_koord_1/36000)*100,2)) , fontsize=18)
plt.show()

In [26]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_1)):
    if i%36000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_1[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_1[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_1[j], color='red',marker='o',markersize=2)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин макс појас износи: {}%'.format(round((trajektorija_koord_2/36000)*100,2)) , fontsize=18)
plt.show()